# Galof

This problem was about some encryption in the finite field $\mathbb F_{2^{255}}$.

In [ ]:
def encrypt(msg, base, key):
	key_1, key_2 = key
	m = bin(int(msg.encode('hex'), 16))[2:]
	assert len(m) <= 256
	f, e = 0, 0
	for b in m[::-1]:
		f += int(b) * x^e
		e += 1
	try:
		h = (key_1 * inverse_mod(f, base) + key_2) % base
	except:
		return 'can\'t encrypt such message, sorry!'
	EXP = h.exponents()
	enc = ''
	for i in range(256):
		if i in EXP:
			enc += '1'
		else:
			enc += '0'
	enc = hex(int(enc[::-1], 2)).lstrip('0x').rstrip('L').zfill(64)
	return enc

### What's it doing?

Well, looks like we have two keys; the function transforms the message into a polynomial $f\in\mathbb F_2[x]$, with degree at most $255$.

Then it performs the encryption, through the formula $h=\frac{k_1}{f}+k_2$, with arithmetic done in the quotient field $\frac{\mathbb F_2[x]}{base}$

In the end, the polynomial $h$ gets converted back to an integer by converting its string of coefficients.

### How do we solve it?

Working in the quotient field, the solution becomes really easy: we only need two couples of (plaintext, ciphertext) to solve the system for $k_1,k_2$: $$\begin{cases}h_1f_1 = k_1+f_1k_2\\h_2f_2 = k_1+f_2k_2\end{cases}$$

Let's code it!

In [8]:
F.<x> = GF(2)[]
base = x^255 + x^199 + 1
factor(base) # it's really irreducible, so we get a field

x^255 + x^199 + 1

In [9]:
K.<t> = QuotientRing(F, F.ideal(base))
t^255+t^199+1

0

In [10]:
# get the polynomial from the ciphertext
def get_h(enc):
    coeff = bin(int(enc,16))[2:]
    p = 0
    for i, c in enumerate(coeff[::-1]):
        p += t^i*int(c)
    return p

# get the polynomial from the plaintext
def get_f(msg):
    m = bin(int(msg.encode('hex'), 16))[2:]
    assert len(m) <= 256
    f, e = 0, 0
    for b in m[::-1]:
        f += int(b) * t^e
        e += 1
    return f
    
h1 = get_h("582c7d41f27a92ad373dec06175f8b0d399bc5d858253bb4b6530c6c608992b1")
h2 = get_h("186f2f1f0eeab93e621b34dbc1fa515b5b74fc761afb9e74a560598844ab9160")
f1 = get_f("uXu2FTYWpCWSXcPwpv4mc0V8nhl2T7")
f2 = get_f("PBFnf2mSWiHUNxMr90KJC6TubsKjU9")

In [11]:
# solve the linear system
k2 = (h1*f1-h2*f2)/(f1-f2)
print k2

k1 = h1*f1-f1*k2
print k1

t^163 + t^99 + t^97 + t^3 + 1
t^163 + t^80 + t^47 + t^9 + 1


### Find the flag!

Now that we have the keys, we can invert the encyption function: if $h=\frac{k_1}{f}+k_2$, then $f=\frac{k_1}{h-k_2}$.

Then we need to convert the polynomial of the flag back into text.

In [12]:
enc_flag = get_h("46c5c88ef8c8f6d49ffc763d56e9cd33176d9aa14c039281d506b834d48c1066")
flag = k1/(enc_flag-k2)
print flag

t^230 + t^225 + t^224 + t^222 + t^217 + t^216 + t^214 + t^212 + t^210 + t^206 + t^202 + t^201 + t^198 + t^197 + t^196 + t^195 + t^193 + t^192 + t^190 + t^186 + t^185 + t^184 + t^182 + t^178 + t^177 + t^173 + t^172 + t^169 + t^166 + t^164 + t^163 + t^162 + t^161 + t^160 + t^158 + t^154 + t^153 + t^149 + t^148 + t^144 + t^142 + t^141 + t^139 + t^138 + t^137 + t^134 + t^131 + t^128 + t^125 + t^124 + t^122 + t^121 + t^120 + t^117 + t^116 + t^113 + t^112 + t^110 + t^108 + t^107 + t^106 + t^105 + t^104 + t^102 + t^97 + t^96 + t^94 + t^93 + t^92 + t^89 + t^86 + t^85 + t^81 + t^80 + t^77 + t^76 + t^72 + t^69 + t^68 + t^65 + t^64 + t^62 + t^60 + t^59 + t^58 + t^57 + t^56 + t^54 + t^53 + t^50 + t^49 + t^45 + t^44 + t^40 + t^38 + t^37 + t^34 + t^32 + t^30 + t^29 + t^27 + t^26 + t^22 + t^21 + t^18 + t^14 + t^13 + t^12 + t^9 + t^8 + t^6 + t^5 + t^4 + t^3 + t^2 + 1


In [13]:
flag_bin = ''
for i in flag.list()[::-1]:
    flag_bin += str(i)   
print flag_bin

h = hex(int(flag_bin,2)).rstrip("L").lstrip("0x")
h.decode('hex')

000000000000000000000000100001101000011010101000100011001111011010001110100011000110010010111110100011000110001011011100100100100110111001100110101111101000011011100100110001100110001001100110101111101100110001100010110010101101100011001000111001101111101


'CCTF{GF2_F1nI73_Crc13_f1elds}'